# Lesson 3: Loading ML Models with Different Data Types

이번 챕터에서는 ML 모델을 여러 자료형으로 불러오는 방법에 대해 배웁니다.

- helper file에서 더미 모델을 불러옵니다.
- `helper.py` 파일은 강의 환경에서 다운로드 할 수 있는데, 다운로드 받아 함께 업로드 해두었습니다.

더미 모델을 인스턴스를 생성하고 그 결과를 확인해 봅니다.

In [1]:
from helper import DummyModel

In [ ]:
model = DummyModel()

In [ ]:
model

- 모델에 존재하는 파라미터의 자료형을 검사하는 함수를 생성합니다.

In [ ]:
def print_param_dtype(model):
    for name, param in model.named_parameters():
        print(f"{name} is loaded in {param.dtype}")

In [ ]:
print_param_dtype(model)

## Model Casting: `float16`

- 허깅페이스의 모델들은 `float32`으로 불러오도록 세팅이 되어 있습니다.
- 여기서는 `float16` 자료형으로 모델을 로드해봅니다.

간단하게, 모델의 인스턴스에 자료형을 바로 선언해주면 됩니다.

In [ ]:
# float 16
model_fp16 = DummyModel().half()

- 파라미터의 자료형을 확인해봅니다. `torch.float16`으로 변경되어야 정상입니다.

In [ ]:
print_param_dtype(model_fp16)

In [ ]:
model_fp16

- 간단한 추론을 해봅니다. 이때 모델은 정수를 입력으로 받기 때문에 (embedding을 위해) `LongTensor`를 생성해줍니다.

In [ ]:
import torch

In [ ]:
dummy_input = torch.LongTensor([[1, 0], [0, 1]])

`float32` 자료형으로는 오류가 발생하지 않고 잘 계산됩니다.

In [ ]:
# inference using float32 model
logits_fp32 = model(dummy_input)

In [ ]:
logits_fp32

그러나 `float16` 자료형으로 변경한 경우 에러가 발생합니다.

In [ ]:
# inference using float16 model
try:
    logits_fp16 = model_fp16(dummy_input)
except Exception as error:
    print("\033[91m", type(error).__name__, ": ", error, "\033[0m")

## Model Casting: `bfloat16`

위에서 언급한 것처럼 허깅페이스에서는 모델을 `float32` 자료형으로 로드하는 것이 default입니다.

이를 억지로 `float16` 자료형으로 로드한다고 하더라도 일반적으로 연산을 지원하지 않습니다.

따라서 일반적으로 사용되는 메모리의 양을 줄이고 싶다면 `bfloat16` 자료형으로 모델을 로드해야 합니다.

#### Note about deepcopy
- `copy.deepcopy`는 원래 모델과 독립적인 객체로 복사해줍니다. 복사본에 대한 수정 결과는 원본에 영향을 주지 않습니다. 자세한 내용을 확인하기 원하는 경우 [링크](https://docs.python.org/3/library/copy.html)를 참고하시기 바랍니다.


In [ ]:
from copy import deepcopy

In [ ]:
model_bf16 = deepcopy(model)

모델의 파라미터를 bf16 자료형으로 변경합니다.

In [ ]:
model_bf16 = model_bf16.to(torch.bfloat16)

In [ ]:
print_param_dtype(model_bf16)

이번에는 에러가 발생하지 않고 정상적으로 연산이 수행된다는 것이 확인됩니다.

In [ ]:
logits_bf16 = model_bf16(dummy_input)

- 이제 `fp32` 자료형으로 연산한 결과와 `bf16` 자료형으로 연산한 결과를 비교해 봅니다.

In [ ]:
mean_diff = torch.abs(logits_bf16 - logits_fp32).mean().item()
max_diff = torch.abs(logits_bf16 - logits_fp32).max().item()

print(f"Mean diff: {mean_diff} | Max diff: {max_diff}")

## Using Popular Generative Models in Different Data Types

- 이미지 캡셔닝 태스크를 위해 [Salesforce/blip-image-captioning-base](https://huggingface.co/Salesforce/blip-image-captioning-base)를 불러옵니다.

#### 샘플 코드를 확인하는 방법
- "Model Card" 탭을 클릭합니다.
- 우측 "<> Use in Transformers"를 클릭하면 모델을 불러오는 샘플 코드가 팝업됩니다.

```Python
# Load model directly
from transformers import AutoProcessor, AutoModelForSeq2SeqLM

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/blip-image-captioning-base")
```

- 예시가 포함된 샘플 코드를 확인하고 싶은 경우, 팝업 맨 아래의 "Read model documentation"를 클릭하세요. 그러면 새 탭이 열립니다.
  https://huggingface.co/docs/transformers/main/en/model_doc/blip#transformers.BlipForConditionalGeneration
- 해당 페이지에서 스크롤을 조금 내려 "parameters" 섹션을 지나면 "Examples:"가 보일 것입니다.

```Python
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "A picture of"

inputs = processor(images=image, text=text, return_tensors="pt")

outputs = model(**inputs)
```

In [ ]:
from transformers import BlipForConditionalGeneration

In [ ]:
model_name = "Salesforce/blip-image-captioning-base"

In [ ]:
model = BlipForConditionalGeneration.from_pretrained(model_name)

아래의 주석을 해제하고 코드를 실행하면 `fp32` 자료형으로 로드된 것이 확인됩니다.

In [ ]:
# inspect the default data types of the model

# print_param_dtype(model)


- 모델의 메모리 사용량을 확인해 봅시다.

In [ ]:
fp32_mem_footprint = model.get_memory_footprint()

In [ ]:
print("Footprint of the fp32 model in bytes: ",
      fp32_mem_footprint)
print("Footprint of the fp32 model in MBs: ", 
      fp32_mem_footprint/1e+6)

- 이번에는 같은 모델을 `bfloat16` 자료형으로 로드합니다.

In [ ]:
model_bf16 = BlipForConditionalGeneration.from_pretrained(
                                               model_name,
                               torch_dtype=torch.bfloat16
)

In [ ]:
bf16_mem_footprint = model_bf16.get_memory_footprint()

메모리 사용량의 차이를 비교해보면 기존 대비 절반으로 줄었다는 것을 알 수 있습니다.

In [ ]:
# Get the relative difference
relative_diff = bf16_mem_footprint / fp32_mem_footprint

print("Footprint of the bf16 model in MBs: ", 
      bf16_mem_footprint/1e+6)
print(f"Relative diff: {relative_diff}")

### Model Performance: `float32` vs `bfloat16`

- 이번에는 실제로 생성 결과가 어떻게 달라지는지 확인해 봅니다.

In [ ]:
from transformers import BlipProcessor

In [ ]:
processor = BlipProcessor.from_pretrained(model_name)

- 이미지를 불러옵니다.

In [ ]:
from helper import load_image, get_generation
from IPython.display import display

img_url = 'https://storage.googleapis.com/\
sfr-vision-language-research/BLIP/demo.jpg'

image = load_image(img_url)
display(image.resize((500, 350)))

In [ ]:
results_fp32 = get_generation(model, 
                              processor, 
                              image, 
                              torch.float32)

In [ ]:
print("fp32 Model Results:\n", results_fp32)

In [ ]:
results_bf16 = get_generation(model_bf16, 
                              processor, 
                              image, 
                              torch.bfloat16)

In [ ]:
print("bf16 Model Results:\n", results_bf16)

### Default Data Type

- 언급한 것처럼 허깅페이스의 트랜스포머 라이브러리는 기본적으로 모델을 `float32` 자료형으로 불러옵니다.
- 모델을 불필요하게 큰 사이즈로 불러오는 것을 방지하기 위해 "default date type"을 변경할 수도 있습니다.

In [ ]:
desired_dtype = torch.bfloat16
torch.set_default_dtype(desired_dtype)

In [ ]:
dummy_model_bf16 = DummyModel()

In [ ]:
print_param_dtype(dummy_model_bf16)

- 모델을 불러왔다면 default data type을 초기화 해줍니다.
- 이 작업은 이미 불러온 모델에 대해 영향을 주지 않습니다.

In [ ]:
torch.set_default_dtype(torch.float32)

In [ ]:
print_param_dtype(dummy_model_bf16)



### Note
- quantization의 아주 간단한 형태로 모델의 파라미터를 더 낮은 자료형으로 불러오는 위 방법을 사용할 수 있습니다. 추론 동안에 모델은 해당 자료형을 기준으로 연산한 결과를 반환하게 됩니다.
- 다음 강의에서는 다른 quantization 기법인 "linear quantization"에 대해 배웁니다. 이는 quantized 모델이 기존의 성능을 잘 유지할 수 있도록 압축된 데이터를 기존의 FP 32 자료형으로 변경하여 계산하는 테크닉을 추론에 적용하는 방식을 뜻합니다.